#  ------------------------------------------------------------------------------------
#                                 **Business Opportunities in Trivandrum**
#  -------------------------------------------------------------------------------------

#### The following project aims to sort and segment the different localities in Trivandrum,Kerala,India according to their business opportunity.The project mainly focus on two business opportunities, Theaters and restaurants. The project utilizes the Wikipedia page for the location details and foursquare API for location venues. Using geocoder, a method to find the latitudes and longitude, the location index of all these places will be acquired. then by means of API calls , the code will acquire venues nearby up to a distance of 3KM. The acquired venues will then be sorted for the two business needs and then using Kmeans clustering cluster the neighborhood into 4 categories.AFter analyzing these, we will decide on places most optimum.
     

**Importing all the required libraries**

In [2]:
import pandas as pd
import numpy as np
import requests
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import matplotlib.cm as cm
import matplotlib.colors as colors
import geocoder
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup

**Next lets pass on the link to the page**

In [5]:
link='https://en.wikipedia.org/wiki/Category:Suburbs_of_Thiruvananthapuram'

**Using the link, the following code will acquire the data in text and pass it ro beautifulsoup which then segregates the useful information**

In [6]:
data = requests.get(link).text
soup = BeautifulSoup(data, 'html.parser')

neighborhoodList = []

for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
tvm = pd.DataFrame({"Neighborhood": neighborhoodList})

tvm.head()

,Neighborhood
0,Aakkulam
1,"Ambalathara, Thiruvananthapuram"
2,Aryankuzhi
3,Attakulangara
4,Attukal


In [7]:
tvm.shape

(67, 1)

Thus there are 67 localities in the city.

Now, Using Geocoder lets find the latitudes and longitudes

In [8]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Kerala,India'.format(neighborhood))# geocoder.goole was non responsing hence used a different locatio coder arcGIS
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in tvm["Neighborhood"].tolist()]

In [10]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [11]:
df_coords.shape

(67, 2)

Here we got back the latitudes and longitudes of all the location.

**Adding these location to their corresponding latitudes and longitudes**

In [12]:
tvm['Latitude']=df_coords['Latitude']
tvm['Longitude'] =df_coords['Longitude']

In [13]:
tvm

,Neighborhood,Latitude,Longitude
0,Aakkulam,11.460200,75.678380
1,"Ambalathara, Thiruvananthapuram",8.450200,76.952210
2,Aryankuzhi,9.926410,76.395320
3,Attakulangara,8.479660,76.949610
4,Attukal,8.469880,76.955140
5,Beemapally,8.457370,76.933170
6,"Chala, Thiruvananthapuram",8.481120,76.954700
7,Chengallur,10.406020,76.299050
8,Connemara Market,8.503310,76.951370
9,East Fort,8.482950,76.944500


**Visualize the map**

Our next task is to visualize the maps,This will allow us to see the location and the neighbourhoods.
By visualising we can avoid locations that are scarecily populated,where a theatre wont be profitable  or restaurant 

In [14]:
address = 'Trivandrum,India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kuala Lumpur, Malaysiae {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kuala Lumpur, Malaysiae 8.5058909, 76.9570481.


In [15]:
City = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, neighborhood in zip(tvm['Latitude'], tvm['Longitude'], tvm['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(City)  
    
display(City)

 From the map we can see that  Pongumoodu and kaithamukku are  overlapping with another location hence this can be removed  so is Aryankuzhi
The following location has been wrongly received hence we will be removing them also 
Vazhayila,
chengallur,
puthoorkonam,
kallatumukku,Aakkulam


In [16]:
tvm.drop([0,2,7,46,60,41,11,12],axis=0,inplace=True) # the index of those places

Since we drop rows , the index will be varied. we will now change the index

In [18]:
tvm.reset_index(drop=True,inplace=True)
tvm.head()

,Neighborhood,Latitude,Longitude
0,"Ambalathara, Thiruvananthapuram",8.45020,76.95221
1,Attakulangara,8.47966,76.94961
2,Attukal,8.46988,76.95514
3,Beemapally,8.45737,76.93317
4,"Chala, Thiruvananthapuram",8.48112,76.95470


In [19]:
City = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, neighborhood in zip(tvm['Latitude'], tvm['Longitude'], tvm['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(City)  
    
display(City)

Visualizing the map again ,we see that the places are now apart

**Onto Foursquare**

Using foursquare Api now we can explore each locality for the location details.

In [20]:
#Declaring credentials 

CLIENT_ID = 'ETV414K5BJF1SOTIWO0BJ4NWK1EYW0BCNKRWYVDZL1LDNYWL' 

CLIENT_SECRET = 'DRIMWO0XDFQZQBQAUEDLINZV33EGNCRYIBV3XHU50ARHGYF0'

VERSION = '20180605'


## **Theatre**

First we will be looking into the location of the theatre
For that I am assuming that a person is willing to travel 5KM for a film, hence we defile the parameters like that

In [21]:
radius = 3000
LIMIT = 100

Now to get the venues in each neighbourhood

In [22]:
venues = []

for lat, long, neighborhood in zip(tvm['Latitude'], tvm['Longitude'], tvm['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

we have a of the places, now we need to convert this list into dataframe

In [23]:
# convert the venues list into a new DataFrame
tvm_venue = pd.DataFrame(venues)

# define the column names
tvm_venue.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(tvm_venue.shape)
tvm_venue.head()

(2379, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"Ambalathara, Thiruvananthapuram",8.4502,76.95221,Hotel Rajila,8.467975,76.932693,Restaurant
1,"Ambalathara, Thiruvananthapuram",8.4502,76.95221,Milma,8.452839,76.950564,Food & Drink Shop
2,"Ambalathara, Thiruvananthapuram",8.4502,76.95221,Meenkada,8.449246,76.948578,Restaurant
3,"Ambalathara, Thiruvananthapuram",8.4502,76.95221,Thiruvananthapuram international airport,8.468655,76.937375,Airport
4,"Ambalathara, Thiruvananthapuram",8.4502,76.95221,Beach and Lake Resort,8.425566,76.958863,Resort


In [24]:
tvm_venue.groupby(['Neighborhood']).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"Ambalathara, Thiruvananthapuram",5,5,5,5,5,5
Attakulangara,66,66,66,66,66,66
Attukal,40,40,40,40,40,40
Beemapally,6,6,6,6,6,6
"Chala, Thiruvananthapuram",64,64,64,64,64,64
Connemara Market,70,70,70,70,70,70
East Fort,69,69,69,69,69,69
Jyothi Nagar,29,29,29,29,29,29
Kamaleswaram,47,47,47,47,47,47


Now to find all the unique category  in them

In [25]:
print('There are {} uniques categories.'.format(len(tvm_venue['VenueCategory'].unique())))
tvm_venue['VenueCategory'].unique()

There are 107 uniques categories.


array(['Restaurant', 'Food & Drink Shop', 'Airport', 'Resort',
       'Multiplex', 'Movie Theater', 'Burger Joint', 'Jewelry Store',
       'Indian Restaurant', 'Temple', 'Fast Food Restaurant',
       'Vegetarian / Vegan Restaurant', 'Arcade', 'Juice Bar', 'Hotel',
       'Shopping Mall', 'Ice Cream Shop', 'Café',
       'Comfort Food Restaurant', 'Department Store',
       'Middle Eastern Restaurant', 'Indie Movie Theater',
       'Clothing Store', 'Chinese Restaurant', 'Beach', 'Food Court',
       'City', 'Accessories Store', 'Student Center', 'Boutique',
       'Pizza Place', 'Bakery', 'Thai Restaurant', 'Train Station',
       'Sporting Goods Shop', 'Stadium', 'Cricket Ground', 'Bus Station',
       'Other Great Outdoors', 'Electronics Store', 'Planetarium',
       'Food Truck', 'History Museum', 'Bistro', 'Historic Site', 'Gym',
       'Snack Place', 'Coffee Shop', 'Food', "Men's Store",
       'Auto Dealership', 'Tennis Stadium', 'Pharmacy',
       'Asian Restaurant', 'Airport 

We can see the different theatres in the city

**One hot encoding**

This is basically to help us in sorting the data,this creates dummy variables for each category a

In [27]:
tvm_code = pd.get_dummies(tvm_venue[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tvm_code['Neighborhoods'] = tvm_venue['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tvm_code.columns[-1]] + list(tvm_code.columns[:-1])
tvm_code = tvm_code[fixed_columns]
print(tvm_code.shape)
tvm_code.head()

(2379, 108)


,Neighborhoods,Accessories Store,Airport,Airport Lounge,Airport Terminal,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bakery,Bar,Beach,Bistro,Bookstore,Botanical Garden,Boutique,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Cafeteria,Café,Chinese Restaurant,City,Clothing Store,Coffee Shop,Comfort Food Restaurant,Cricket Ground,Department Store,Design Studio,Dessert Shop,Diner,Dog Run,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Grocery Store,Gym,Halal Restaurant,Harbor / Marina,Historic Site,History Museum,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Jewelry Store,Juice Bar,Kerala Restaurant,Kids Store,Lake,Lighthouse,Lounge,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Neighborhood,Optical Shop,Other Great Outdoors,Park,Pharmacy,Pizza Place,Planetarium,Plaza,Resort,Restaurant,River,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Ski Area,Snack Place,South Indian Restaurant,Sporting Goods Shop,Stables,Stadium,Student Center,Surf Spot,Tea Room,Temple,Tennis Stadium,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Warehouse Store,Women's Store,Zoo
0,"Ambalathara, Thiruvananthapuram",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Ambalathara, Thiruvananthapuram",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Ambalathara, Thiruvananthapuram",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Ambalathara, Thiruvananthapuram",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Ambalathara, Thiruvananthapuram",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
tvm_code_gp = tvm_code.groupby(["Neighborhoods"]).mean().reset_index()

print(tvm_code_gp.shape)
tvm_code_gp.head()

(59, 108)


,Neighborhoods,Accessories Store,Airport,Airport Lounge,Airport Terminal,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bakery,Bar,Beach,Bistro,Bookstore,Botanical Garden,Boutique,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Cafeteria,Café,Chinese Restaurant,City,Clothing Store,Coffee Shop,Comfort Food Restaurant,Cricket Ground,Department Store,Design Studio,Dessert Shop,Diner,Dog Run,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Grocery Store,Gym,Halal Restaurant,Harbor / Marina,Historic Site,History Museum,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Jewelry Store,Juice Bar,Kerala Restaurant,Kids Store,Lake,Lighthouse,Lounge,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Neighborhood,Optical Shop,Other Great Outdoors,Park,Pharmacy,Pizza Place,Planetarium,Plaza,Resort,Restaurant,River,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Ski Area,Snack Place,South Indian Restaurant,Sporting Goods Shop,Stables,Stadium,Student Center,Surf Spot,Tea Room,Temple,Tennis Stadium,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Warehouse Store,Women's Store,Zoo
0,"Ambalathara, Thiruvananthapuram",0.000000,0.2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.200000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.2,0.400000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,Attakulangara,0.015152,0.0,0.0,0.0,0.0,0.030303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,0.0,0.015152,0.0,0.0,0.0,0.015152,0.0,0.0,0.015152,0.000,0.0,0.0,0.060606,0.015152,0.015152,0.045455,0.0,0.015152,0.015152,0.030303,0.0,0.0,0.0,0.0,0.000000,0.0,0.045455,0.0,0.0,0.0,0.030303,0.015152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,0.0,0.015152,0.151515,0.015152,0.0,0.0,0.015152,0.015152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015152,0.0,0.045455,0.030303,0.0,0.0,0.000000,0.0,0.0,0.015152,0.0,0.0,0.0,0.060606,0.0,0.0,0.0,0.0,0.0,0.045455,0.0,0.0,0.0,0.015152,0.0,0.015152,0.015152,0.0,0.0,0.015152,0.0,0.015152,0.015152,0.015152,0.0,0.0,0.0
2,Attukal,0.025000,0.0,0.0,0.0,0.0,0.025000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050000,0.0,0.025000,0.0,0.0,0.0,0.050000,0.0,0.0,0.025000,0.025,0.0,0.0,0.000000,0.025000,0.000000,0.050000,0.0,0.025000,0.000000,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.050000,0.0,0.0,0.0,0.025000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050000,0.0,0.000000,0.150000,0.025000,0.0,0.0,0.025000,0.025000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.025000,0.050000,0.0,0.0,0.000000,0.0,0.0,0.025000,0.0,0.0,0.0,0.025000,0.0,0.0,0.0,0.0,0.0,0.050000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.025000,0.0,0.0,0.025000,0.0,0.025000,0.000000,0.025000,0.0,0.0,0.0
3,Beemapally,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.166667,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.333333,0.000000,0.0,0.0,0.000000,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.166667,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.166667,0.000000,0.000000,0.0,0.0,0.0
4,"Chala, Thiruvananthapuram",0.015625,0.0,0.0,0.0,0.0,0.031250

From this sorted list we will be listing out the needed theatre dataframe
theatre in city are named differently like multiplees,Indian theatre, 
Another thing to remember is that the shopping malls also have theatres, we should include them in our category

In [29]:
tvm_theatre=tvm_code_gp[['Neighborhoods','Multiplex', 'Movie Theater','Shopping Mall']]
tvm_theatre.head()

,Neighborhoods,Multiplex,Movie Theater,Shopping Mall
0,"Ambalathara, Thiruvananthapuram",0.000000,0.000000,0.000000
1,Attakulangara,0.030303,0.045455,0.045455
2,Attukal,0.050000,0.025000,0.050000
3,Beemapally,0.000000,0.000000,0.000000
4,"Chala, Thiruvananthapuram",0.031250,0.031250,0.031250


**Sorting these Locations**

These locations needs to be sorted into clusters.Here I use a k means approach of unsupervised clustering. We will divide the city spots into 4 clusters

In [66]:
k = 4
tvm_clustering = tvm_theatre.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(tvm_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]
tvm_city = tvm_theatre.copy()

# add clustering labels
tvm_city["Cluster Labels"] = kmeans.labels_
tvm_city.columns=['Neighborhood','Multiplex','Movie Theatre','Shopping Mall','Cluster Labels']
tvm_city.head()

,Neighborhood,Multiplex,Movie Theatre,Shopping Mall,Cluster Labels
0,"Ambalathara, Thiruvananthapuram",0.000000,0.000000,0.000000,2
1,Attakulangara,0.030303,0.045455,0.045455,0
2,Attukal,0.050000,0.025000,0.050000,0
3,Beemapally,0.000000,0.000000,0.000000,2
4,"Chala, Thiruvananthapuram",0.031250,0.031250,0.031250,0


we need to add these clustered data with their latitudes and longitudes for the visualization

In [67]:
tvm_city = tvm_city.join(tvm.set_index("Neighborhood"), on="Neighborhood")
k=4
print(tvm_city.shape)
tvm_city.head()

(59, 7)


,Neighborhood,Multiplex,Movie Theatre,Shopping Mall,Cluster Labels,Latitude,Longitude
0,"Ambalathara, Thiruvananthapuram",0.000000,0.000000,0.000000,2,8.45020,76.95221
1,Attakulangara,0.030303,0.045455,0.045455,0,8.47966,76.94961
2,Attukal,0.050000,0.025000,0.050000,0,8.46988,76.95514
3,Beemapally,0.000000,0.000000,0.000000,2,8.45737,76.93317
4,"Chala, Thiruvananthapuram",0.031250,0.031250,0.031250,0,8.48112,76.95470


### **Visualizing the clusters**

In [68]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# color scheme for the clusters
x = np.arange(k)
ys = [i+x+(i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))# color maping to an external trainbow package
rainbow = [colors.rgb2hex(i) for i in colors_array]# converting thr rgb values to heaxa decimal values

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tvm_city['Latitude'], tvm_city['Longitude'], tvm_city['Neighborhood'], tvm_city['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In the above plot,we can see that the city is divided into several clusters using different colors


**Lets analyze each cluster individually**

**Cluster 1**

In [37]:
tvm_city.loc[tvm_city['Cluster Labels']==0]

,Neighborhood,Multiplex,Movie Theatre,Shopping Mall,Cluster Labels,Latitude,Longitude
1,Attakulangara,0.030303,0.045455,0.045455,0,8.479660,76.949610
2,Attukal,0.050000,0.025000,0.050000,0,8.469880,76.955140
4,"Chala, Thiruvananthapuram",0.031250,0.031250,0.031250,0,8.481120,76.954700
5,Connemara Market,0.028571,0.042857,0.042857,0,8.503310,76.951370
6,East Fort,0.028986,0.043478,0.043478,0,8.482950,76.944500
8,Kamaleswaram,0.085106,0.042553,0.042553,0,8.466110,76.944830
10,Kannammoola,0.035088,0.070175,0.052632,0,8.503353,76.931743
11,Karamana,0.062500,0.031250,0.031250,0,8.470490,76.965050
15,Konchiravila,0.125000,0.062500,0.062500,0,8.462950,76.952970
17,Kudappanakunnu,0.000000,0.000000,0.117647,0,8.549790,76.954610


**Cluster 2**

In [40]:
tvm_city.loc[tvm_city['Cluster Labels']==1]

,Neighborhood,Multiplex,Movie Theatre,Shopping Mall,Cluster Labels,Latitude,Longitude
38,Pothencode,0.0,0.5,0.0,1,8.60367,76.89230
57,Vettuthura,0.0,0.5,0.0,1,8.59091,76.82206


**Cluster 3**

In [42]:
tvm_city.loc[tvm_city['Cluster Labels']==2]

,Neighborhood,Multiplex,Movie Theatre,Shopping Mall,Cluster Labels,Latitude,Longitude
0,"Ambalathara, Thiruvananthapuram",0.000000,0.000000,0.000000,2,8.45020,76.95221
3,Beemapally,0.000000,0.000000,0.000000,2,8.45737,76.93317
7,Jyothi Nagar,0.000000,0.034483,0.000000,2,8.54032,76.89261
9,Kaniyapuram,0.000000,0.000000,0.000000,2,8.59160,76.88254
12,Kariavattom,0.000000,0.032258,0.000000,2,8.56955,76.89037
13,Kazhakoottam,0.000000,0.031250,0.000000,2,8.56840,76.86893
14,Kesavadasapuram,0.000000,0.000000,0.047619,2,8.52961,76.93641
16,Kowdiar,0.000000,0.020000,0.040000,2,8.52318,76.95671
18,Kuzhivila,0.000000,0.000000,0.000000,2,8.52648,76.89241
20,Mannanmoola,0.000000,0.000000,0.040816,2,8.52984,76.97703


**Custer 4**

In [45]:
tvm_city.loc[tvm_city['Cluster Labels']==3]

,Neighborhood,Multiplex,Movie Theatre,Shopping Mall,Cluster Labels,Latitude,Longitude
35,Peyad,0.25,0.0,0.0,3,8.51439,77.0193
50,Valiyavila,0.25,0.0,0.0,3,8.50582,77.0004


### **Observation**

It can be observed that the cluster 1 has high concentration of movie theatres, this can be seen in the map also that these are the sites in he heart of the city, hence this also points out that the city center is having sufficient theatres. Coming to cluster 2,it lies far from city centre but the cluster has seen that here are theatres but low in number
cluster 3 lies throughout the city, has low to none of the amount available theatres.Cluster 4 si similar to cluster 2 has multiplexes. 
From a business point of view it is better to open a theatre in the places marked in cluster 3

## **Restaurants**
Now lets take a look at locations for a hotel.This can be done in the same way as the above
From the one hot encoded dataframe, lets look for restaurants

In [48]:
tvm_code_gp

,Neighborhoods,Accessories Store,Airport,Airport Lounge,Airport Terminal,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bakery,Bar,Beach,Bistro,Bookstore,Botanical Garden,Boutique,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Cafeteria,Café,Chinese Restaurant,City,Clothing Store,Coffee Shop,Comfort Food Restaurant,Cricket Ground,Department Store,Design Studio,Dessert Shop,Diner,Dog Run,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Grocery Store,Gym,Halal Restaurant,Harbor / Marina,Historic Site,History Museum,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Jewelry Store,Juice Bar,Kerala Restaurant,Kids Store,Lake,Lighthouse,Lounge,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Neighborhood,Optical Shop,Other Great Outdoors,Park,Pharmacy,Pizza Place,Planetarium,Plaza,Resort,Restaurant,River,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Ski Area,Snack Place,South Indian Restaurant,Sporting Goods Shop,Stables,Stadium,Student Center,Surf Spot,Tea Room,Temple,Tennis Stadium,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Warehouse Store,Women's Store,Zoo
0,"Ambalathara, Thiruvananthapuram",0.000000,0.200000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.00000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.400000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Attakulangara,0.015152,0.000000,0.0000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.00000,0.015152,0.000000,0.000000,0.0,0.015152,0.00000,0.0,0.015152,0.000000,0.000000,0.000000,0.060606,0.015152,0.015152,0.045455,0.000000,0.015152,0.015152,0.030303,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.045455,0.000000,0.00000,0.000000,0.030303,0.015152,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.015152,0.151515,0.015152,0.000000,0.00,0.015152,0.015152,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.015152,0.000000,0.045455,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.015152,0.000000,0.000000,0.000000,0.060606,0.00,0.000000,0.000000,0.000000,0.000000,0.045455,0.00,0.000000,0.000000,0.015152,0.000000,0.015152,0.015152,0.00,0.000000,0.015152,0.000000,0.015152,0.015152,0.015152,0.000000,0.000000,0.000000
2,Attukal,0.025000,0.000000,0.0000,0.000000,0.000000,0.025000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.00000,0.025000,0.000000,0.000000,0.0,0.050000,0.00000,0.0,0.025000,0.025000,0.000000,0.000000,0.000000,0.025000,0.000000,0.050000,0.000000,0.025000,0.000000,0.050000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.050000,0.000000,0.00000,0.000000,0.025000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.150000,0.025000,0.000000,0.00,0.025000,0.025000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.025000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02500

We will choose Fast food restaurant and restaurant column titles for proceeding

In [49]:
tvm_RR=tvm_code_gp[['Neighborhoods','Restaurant', 'Fast Food Restaurant']]
tvm_RR.head()

,Neighborhoods,Restaurant,Fast Food Restaurant
0,"Ambalathara, Thiruvananthapuram",0.400000,0.000000
1,Attakulangara,0.060606,0.045455
2,Attukal,0.025000,0.050000
3,Beemapally,0.000000,0.000000
4,"Chala, Thiruvananthapuram",0.062500,0.046875


**Sorting**

Now like previous step ,we can cluster them
here we will cluster them into 3 groups

In [53]:
k = 3
tvm_Rclustering = tvm_RR.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(tvm_Rclustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]
tvm_Rcity = tvm_RR.copy()

# add clustering labels
tvm_Rcity["Cluster Labels"] = kmeans.labels_
tvm_Rcity.columns=['Neighborhood','Restaurant','Fast Food Restaurant','Cluster Labels']
tvm_Rcity.head()

,Neighborhood,Restaurant,Fast Food Restaurant,Cluster Labels
0,"Ambalathara, Thiruvananthapuram",0.400000,0.000000,0
1,Attakulangara,0.060606,0.045455,1
2,Attukal,0.025000,0.050000,1
3,Beemapally,0.000000,0.000000,1
4,"Chala, Thiruvananthapuram",0.062500,0.046875,1


In [54]:
tvm_Rcity = tvm_Rcity.join(tvm.set_index("Neighborhood"), on="Neighborhood")

print(tvm_Rcity.shape)
tvm_Rcity.head()

(59, 6)


,Neighborhood,Restaurant,Fast Food Restaurant,Cluster Labels,Latitude,Longitude
0,"Ambalathara, Thiruvananthapuram",0.400000,0.000000,0,8.45020,76.95221
1,Attakulangara,0.060606,0.045455,1,8.47966,76.94961
2,Attukal,0.025000,0.050000,1,8.46988,76.95514
3,Beemapally,0.000000,0.000000,1,8.45737,76.93317
4,"Chala, Thiruvananthapuram",0.062500,0.046875,1,8.48112,76.95470


**Lets visualize the clusters**

In [56]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# color scheme for the clusters
x = np.arange(k)
ys = [i+x+(i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))# color maping to an external trainbow package
rainbow = [colors.rgb2hex(i) for i in colors_array]# converting thr rgb values to heaxa decimal values

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tvm_Rcity['Latitude'], tvm_Rcity['Longitude'], tvm_Rcity['Neighborhood'], tvm_Rcity['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Lets analyze each cluster**

**Cluster 1**

In [60]:
tvm_Rcity.loc[tvm_Rcity['Cluster Labels']==0]

,Neighborhood,Restaurant,Fast Food Restaurant,Cluster Labels,Latitude,Longitude
0,"Ambalathara, Thiruvananthapuram",0.400000,0.0,0,8.45020,76.95221
37,Poonthura,0.333333,0.0,0,8.44921,76.94815


**Cluster 2**

In [61]:
tvm_Rcity.loc[tvm_Rcity['Cluster Labels']==1]

,Neighborhood,Restaurant,Fast Food Restaurant,Cluster Labels,Latitude,Longitude
1,Attakulangara,0.060606,0.045455,1,8.479660,76.949610
2,Attukal,0.025000,0.050000,1,8.469880,76.955140
3,Beemapally,0.000000,0.000000,1,8.457370,76.933170
4,"Chala, Thiruvananthapuram",0.062500,0.046875,1,8.481120,76.954700
5,Connemara Market,0.042857,0.071429,1,8.503310,76.951370
6,East Fort,0.043478,0.043478,1,8.482950,76.944500
7,Jyothi Nagar,0.068966,0.000000,1,8.540320,76.892610
8,Kamaleswaram,0.021277,0.042553,1,8.466110,76.944830
9,Kaniyapuram,0.000000,0.076923,1,8.591600,76.882540
10,Kannammoola,0.052632,0.035088,1,8.503353,76.931743


**Cluster 3**

In [62]:
tvm_Rcity.loc[tvm_Rcity['Cluster Labels']==2]

,Neighborhood,Restaurant,Fast Food Restaurant,Cluster Labels,Latitude,Longitude
14,Kesavadasapuram,0.095238,0.031746,2,8.52961,76.93641
22,Nalanchira,0.111111,0.027778,2,8.54336,76.93612
23,Nemom,0.200000,0.000000,2,8.45221,76.97524
31,Pattom,0.096154,0.038462,2,8.51667,76.93979
39,Pulayanarkotta,0.138889,0.027778,2,8.52056,76.91681
42,Sreekaryam,0.142857,0.000000,2,8.54886,76.91659
47,Thumba,0.137931,0.000000,2,8.53521,76.86716
48,Ulloor,0.117647,0.029412,2,8.53789,76.92615
58,Vizhinjam,0.125000,0.031250,2,8.38566,76.99091


### **OBSERVATION** - **1.Restaurant**

Here  the 3 clusters are cluster 1 has large concentration of restaurants hence in cluster 1 we should not open a restaurant,In cluster 2 we see that neither restaurant nor fast food restaurants are common, means they are rare, these even include the city centre.Cluster 3 has good amount of restaurants and fast food chains but lower than cluster 1. So best option will be to open restaurants in areas in cluster 2

### **Theatre**

It can be observed that the cluster 1 has high concentration of movie theatres, this can be seen in the map also that these are the sites in he heart of the city, hence this also points out that the city center is having sufficient theatres. Coming to cluster 2,it lies far from city centre but the cluster has seen that here are theatres but low in number
cluster 3 lies throughout the city, has low to none of the amount available theatres.Cluster 4 si similar to cluster 2 has multiplexes. 
From a business point of view it is better to open a theatre in the places marked in cluster 3